In [ ]:
# Import dependencies
from pprint import pprint
import pandas as pd 
import requests
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
from sklearn import datasets

import gmaps
import os
import json
import time
from config import g_key

In [ ]:
merged_clean = pd.read_csv('data/yelp_inspection_merge.csv')
merged_clean.head()


In [ ]:
pf = merged_clean["Year"].unique()
pf

# Failed

In [ ]:
#DataFrame for Restaurant with Failed Status
failed_df = merged_clean.loc[merged_clean["Results"] == "Fail"]

In [ ]:
fail = pd.DataFrame(failed_df, columns = ['name','review_count','category','category_title','Risk', 'price_integer',
                                                'rating', 'Year', 'zip code_y','address1','price','Inspection Type','Results','Violations','Latitude','Longitude'])
#fail.head(5)


In [ ]:
#create link for gmaps
gmaps.configure(api_key = g_key)

In [ ]:
# configure gmaps for lat/lng vs rating based on . Store latitude and longitude in locations
locations_f = fail[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = fail["rating"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations_f, weights=rating, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=50)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
#configure a map with all the points
figure = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure)

#Add markers
markers = gmaps.marker_layer(locations_f)
# Add the layer to the map
fig.add_layer(markers)
fig

In [ ]:
f = failed_df["Year"].unique()
f

# Passing DataFrame

In [ ]:
#DataFrame for Restaurant with Passed Status
passed_df = merged_clean.loc[merged_clean["Results"] == "Pass"]
pass_df = pd.DataFrame(passed_df, columns = ['name','review_count','category','category_title','Risk', 'price_integer',
                                                'rating', 'Year', 'zip code_y','address1','price','Inspection Type','Results','Violations','Latitude','Longitude'])
#pass_df.head(5)

Heat Map Rating with Lat/Lng: use Latitude, Longitude and Rating as weight

In [ ]:
#configure gmaps for lat/lng vs rating

# Store latitude and longitude in locations
locations = pass_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = pass_df["rating"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=600,
                                 point_radius=100)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Marker Map of Pass: use Latitude, Longitude

In [ ]:
#configure a map with all the points
locations_p = pass_df[["Latitude", "Longitude"]]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

#Add markers
markers = gmaps.marker_layer(locations_p)
# Add the layer to the map
fig.add_layer(markers)
fig

Price vs Year ScatterPlot

In [ ]:
pass_df['rating'].replace('', np.nan, inplace=True)
pass_df.dropna(subset=['rating'], inplace=True)
p = pass_df["Year"].unique()
p

In [ ]:

x_values = pass_df["Year"]
y_values = pass_df["rating"]


In [ ]:
# Calculate the correlation coefficient and linear regression model 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#create the scatter plot
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(5.8,0.8),fontsize=15,color="red")
plt.xlabel("Year")
plt.ylabel("Rating")
plt.show()

# Pass vs Fail in Bar Chart

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
Passed = pass_df['name'].count()
Failed = failed_df['name'].count()
print(Passed)
print(Failed)

In [ ]:
result = ["Pass", "Fail"] 
count = [441,184]
res_df = pd.DataFrame({"Result": result,"Count": count})
res_df


In [ ]:

# Generate a pie plot showing the distribution of pass vs fail

colors = ["lightblue","orange"]

plot = res_df.plot.pie(y="Count", labels=("Pass","Fail"), figsize=(7,5), colors = colors, autopct="%1.1f%%",
                             startangle=140,)
# Create axes which are equal so we have a perfect circle
plt.axis("equal")
# Save an image of our chart and print the final product to the screen
plt.savefig('images/pie_pass_fail.png')
plt.show()

In [ ]:
pass_df

# pie chart for passing vs rating

In [ ]:
pass_df['bins'] = pd.cut(pass_df['rating'],bins=[0, 2, 3.5, 5], labels=['Poor', 'Okay', 'Great'])
P_df = pass_df.groupby(['rating', 'bins']).size().unstack(fill_value=0)
print(P_df)


In [ ]:
P_df.plot.pie(subplots=True,figsize=(20, 20))
plt.savefig('images/pie_passing_rating.png')

In [ ]:
P = pass_df.groupby('bins').size()
P.plot.pie(figsize=(5,5))
plt.savefig('images/pie_all_passing_rating.png')

# Pie chart for failing vs rating

In [ ]:
fail['Binned'] = pd.cut(fail['rating'], bins=[0, 2, 3.5, 5], labels=['Poor', 'Okay', 'Great'])
F_df = fail.groupby(['rating', 'Binned']).size().unstack(fill_value=0)
print (F_df)


In [ ]:
F_df.plot.pie(subplots=True,figsize=(20, 20))
plt.savefig('images/pie_failing_rating.png')

In [ ]:
F = fail.groupby('Binned').size()
F.plot.pie(figsize=(5,5))
plt.savefig('images/pie_all_failing_rating.png')

In [ ]:
bad_good = fail.loc[fail["Binned"] == "Great" ]
bad_good['name'].unique()

In [ ]:
topo = fail.loc[fail["name"] == "Topo Gigio Ristorante"]
topo

In [ ]:
price_bad = bad_good.loc[bad_good['price_integer'] > 3.0]
price_bad

In [ ]:
price_bad['name'].unique()